In [66]:
#https://chromedriver.chromium.org/downloads
#크롬 버전확인 후 버전에 맞는 Driver를 다운로드 받아 C:\Windows\System32에 복사

import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException

import time
import atexit
import os
import pandas as pd
from glob import glob

In [28]:
#전역 변수 선언
TIMEOUT = 30 #10초
ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
DOWNLOAD_DIR = f'{ROOT_DIR}\\download_2'
print(DOWNLOAD_DIR)

#사이트 주소
URL = 'http://terms.tta.or.kr/dictionary/searchFirstList.do'

/Users/mingyeseon/development/workspaces/python/weekend/RPA\download_2


In [29]:
#chrome webdriver를 생성한다.
# - download_path: 파일을 다운로드 받을 위치
# - headless: True이면 브라우저 화면 없이 background로 처리
def create_chrome(download_path, headless=False) :
    options = ChromeOptions()
    if headless:
        options.add_argument('headless')
        options.add_argument('--disable-gpu')
    options.add_experimental_option('prefs', {
        'download.default_directory' : download_path,
        'download.prompt_for_download' : False
    })

    chrome = Chrome(chrome_options = options)

    if headless :
        _enable_download_on_chrome_headless(chrome, download_path)

    #프로세스 종료시 크롬을 종료시키는 로직
    atexit.register(_close_chrome(chrome))
    
    return chrome

#headless=True 일때 다운로드 안되는(크롬 보안문제) 문제 해결
def _enable_download_on_chrome_headless(browser, download_dir) :
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

    params = {
        'cmd': 'Page.setDownloadBehavior',
        'params': {
            'behavior': 'allow',
            'downloadPath': download_dir
        }
    }
    browser.execute("send_command", params)

#크롬 종료~
def _close_chrome(chrome) :
    def close() :
        chrome.close()
    return close

def chrome_wait(wait, condition):
    try:
        element = wait.until(condition)
        print("[DEBUG] 로딩 완료")
        return element
    except TimeoutException:
        print(f"[DEBUG] 페이지 로딩 시간 초과")

In [70]:
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)

browser.get(URL)

browser.execute_script("fn_firstWord('ㄱ')")



In [47]:
#카테고리 동적으로 접근하기
from urllib import parse

elements = browser.find_elements_by_xpath('//p[@class="korean" or @class="english"]/a')
goto = elements[5].get_attribute('href')
goto = parse.unquote(goto)
browser.execute_script(goto)

In [59]:
#페이지 동적으로 접근하기
import re

list_cnt_per_page = 10
total_page = 0

value = browser.find_element_by_xpath('//li/div[starts-with(text(), "[총")]').text
total_page = int(re.findall("\d+", value)[0])
print(total_page)


[총 2108 건]
2108


In [72]:
def find_word(e) :
    value = e.get_attribute('value')
    browser.execute_script(f'javascript:fn_dictionaryView("{value}")')
    time.sleep(0.5)
    word = browser.find_element_by_xpath('//div[@id="cont"]/dl/dt[1]').text
    synonym = ""
    try :
        synonym = browser.find_element_by_xpath('//div[@id="cont"]/dl/dt[2]').text
    except NoSuchElementException :
        print("No Such Element..")
    
    description = browser.find_element_by_xpath('//div[@id="cont"]/dl/dd[1]/div').get_attribute('innerHTML') #내용 중 HTML 태그 삭제 필요
    description = re.sub('<.+?>', '', description, re.I|re.S)
    #synonym은 없을 수 있다.
    return word, synonym, description

In [63]:
description = """이미 알려진 바이러스를 부트 섹터, 메모리, 디스크의 마스터 부트, 파일 등에서 검색하는 바이러스 방역 기본 프로그램. 바이러스 검색기라고 불린다. 바이러스 정보를 보관하고 있는 <a href="/dictionary/dictionaryView.do?subject=%EC%9E%90%EB%A3%8C+%ED%8C%8C%EC%9D%BC" target="iframe">자료 파일</a>에 기록되지 않은 새로운 바이러스나 변종 바이러스는 검색되지 않으므로 정보 파일을 수시로 갱신(update)하여야 한다."""
re.sub('<.+?>', '', description, re.I|re.S)

'이미 알려진 바이러스를 부트 섹터, 메모리, 디스크의 마스터 부트, 파일 등에서 검색하는 바이러스 방역 기본 프로그램. 바이러스 검색기라고 불린다. 바이러스 정보를 보관하고 있는 자료 파일에 기록되지 않은 새로운 바이러스나 변종 바이러스는 검색되지 않으므로 정보 파일을 수시로 갱신(update)하여야 한다.'

In [73]:
for page in range(1, 3) : #page
    browser.execute_script(f'javascript:fnNavigate({page});')
    print(f'[INFO]{page} 접근')
    time.sleep(1)
    elements = browser.find_elements_by_name('first_seq')
    for element in elements :
        word, synonym, description = find_word(element)
        print(f"{word} ////// {synonym} ////// {description} ")
        #word 핸들링

#수집된 데이터 DataFrame으로 만든 후 CSV 파일에 저장

[INFO]1 접근
No Such Element..
가간섭성, 可干涉性, coherence, coherency //////  ////// ①전자파(電磁波)가 파장이 같은 가지런한 정현파의 집합인 상태. 통신에 이용하는 전파는 모두 가간섭성 파이지만, 일반의 빛은 그렇지 않다. 특별히 가간섭성 광을 발생시켜 이용하는 것으로는 레이저나 홀로그래피(holography)가 있다. ②시간적으로나 공간적으로 또는 시간적/공간적 양쪽으로 분리된 점에서의 전자계(電磁界) 간의 상관관계. 
가간섭성 광, 可干涉性光, coherent ray ////// 동의어 : 코히런트 광통신 ////// 복수 광파(光波)의 위상과 주파수가 같아서 공간적으로나 시간적으로 일치할 때의 광파. 통신에 이용되는 광은 가간섭성 광이어야 하나 일반 광은 그렇지 않다. 가간섭성의 발진기를 사용하여 특별히 가간섭성 광을 생성시켜 이용하는 것이 레이저나 홀로그래피(holography)이다. 
가간섭성 광통신, 可干涉性光通信, coherent optical communication ////// 동의어 : 코히런트 광통신 ////// 빛의 위상 정보를 이용하여 신호를 전송하는 통신 방식. 가간섭성(coherent)이란 단일 주파수 스펙트럼을 가지며, 위상이 동기된 균일한 정현파를 말한다. 초고속이고 대용량인 기가비트 전송에 주로 이용된다. 
가간섭성 광파 전송, 可干涉性光波傳送, coherent light wave transmission ////// 동의어 : 코히런트 광파 전송 ////// 전자파와 마찬가지로 광파의 가간섭성을 이용하여 빛의 주파수나 위상에 신호를 실어서 전송하는 방식. 이 경우에 가간섭성(코히런트)이란 주파수가 일정하여 위상이 시간적, 공간적으로 일정한 빛의 파, 즉 가간섭성 광파를 정보를 싣는 파로 이용한다는 의미이다. 빛의 강약에 따른 종래의 강도 변조 광섬유 전송 방식보다 고속, 대용량 전송이 가능하다. 광대역 종합 정보 통신망(B-ISDN)이나 가입자의 가정까지 광섬유를 포설하는 광

In [74]:
browser.close()

In [ ]:
def prepare_download() :
    print("[INFO] 사이트 이동")
    browser.get(URL)

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.presence_of_element_located((By.CLASS_NAME, 'menu')))

    #로그인 버튼 클릭
    print("[INFO] 용어목록 클릭")
    browser.execute_script("fn_firstWord('ㄱ')")

  
#---------------------------------------------------------------------------------
    
    #ID 입력
    idTag = browser.find_element_by_id('id01')
    idTag.clear()
    idTag.send_keys(USER)

    #패스워드 입력 후 엔터
    pwTag = browser.find_element_by_id('id02')
    pwTag.clear()
    pwTag.send_keys(PASSWD)
    pwTag.send_keys(Keys.RETURN)

    print("[INFO] 로그인 정보 입력 후 로그인 시작")

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btnSearch')))


    #검색조건 입력 후 검색버튼 클릭
    print("[INFO] 검색시작")
    search = browser.find_element_by_id('query')
    search.clear()
    search.send_keys(SEARCH_WORD)
    search.send_keys(Keys.RETURN)

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//li[@class="tabM1 " or @class="tabM1 on"]/a')))

    print("[INFO] 국내학술논문 클릭")
    #국내학술논문 클릭
    browser.find_element_by_xpath('//li[@class="tabM1 " or @class="tabM1 on"]/a').click()

    print("[INFO] 100개씩 출력으로 조회")
    chrome_wait(wait, EC.presence_of_element_located((By.ID, 'sortSelect2_top')))
    #100개씩 출력 선택 및 조회버튼 클릭
    browser.find_element_by_xpath('//label[@for="sortSelect2_top"]').click()
    browser.find_element_by_xpath('//div[@class="optList on"]/ul/li/a[text()="100개씩 출력"]').click()
    browser.find_element_by_css_selector('button.listSearch').click()

    print("[INFO] 작성언어 한국어 선택 및 실행")
    chrome_wait(wait, EC.presence_of_element_located((By.ID, 're_a_kor70')))
    browser.find_element_by_xpath('//label[@for="re_a_kor70"]').click()
    time.sleep(0.5)
    browser.find_element_by_class_name("btnAction").click()

In [ ]:
#파일 다운로드 함수
def download_abstract(pageIndex, wait) : 

    print("[INFO] --전체목록 선택")
    chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//div[@class="resultTop1"]/ul/li[1]/a')))

    #전체항목 선택
    browser.find_element_by_xpath('//label[@for="allchk3"]/span').click()

    print("[INFO] --내보내기 버튼 클릭")
    #내보내기 버튼 클릭
    browser.find_element_by_xpath('//div[@class="resultTop1"]/ul/li[1]/a').click()

    #내보내기 팝업으로 포커스를 이동
    for handler in browser.window_handles:
        if handler != browser.current_window_handle :
            browser.switch_to_window(handler)
            break

    #페이지 로드될 때까지 기다림 
    print("[INFO] --다운로드 조건 선택")
    chrome_wait(wait, EC.element_to_be_clickable((By.CLASS_NAME, "btnType1")))

    #다운로드 조건 선택
    browser.find_element_by_xpath('//label[@for="radio-3"]').click()
    browser.find_element_by_xpath('//label[@for="radio-8"]').click()
    browser.find_element_by_xpath('//label[@for="radio-12"]').click()

    #다운로드 버튼 클릭
    print("[INFO] --다운로드 시작")
    browser.execute_script("javascript:f_submit();")

    #다운로드가 완료되었는지 기다림
    fileName = os.path.join(DOWNLOAD_DIR, "myCabinetExcelData.xls")
    accumulateTime = 1;
    while not os.path.exists(fileName) :
        time.sleep(1)

    #파일이 존재하는지 확인하고 파일명 변경
    newFileName = os.path.join(DOWNLOAD_DIR, f"excel\\abstract_{pageIndex + 1}.xls")
    if(os.path.isfile(fileName) and os.path.getsize(fileName) > 0) :
        os.rename(fileName, newFileName)
        print("[INFO] --다운로드 파일 확인 완료")

        #xls to csv 파일 변경
        header = False
        if pageIndex == 0 : 
            header = True
        csvFilename = os.path.join(DOWNLOAD_DIR, f"csv\\abstract_{pageIndex + 1}.csv")
        read_file = pd.read_excel(newFileName)
        read_file.to_csv(csvFilename, index = None, header = header)
        print("[INFO] --CSV 파일 변환 완료")

    else :
        print(f"[INFO] --다운로드 파일 확인 실패 [{newFileName}]")
    
    #팝업 닫기
    browser.close()
    #원래 페이지로 포커스 이동
    browser.switch_to_window(browser.window_handles[0]) 
    
    time.sleep(1)
    #전체 선택 해제
    browser.find_element_by_xpath('//label[@for="allchk3"]/span').click()

In [ ]:
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)

In [ ]:
prepare_download()

In [ ]:
## 페이지별로 다운로드 반복 구간
#한페이지: 100개
#전체자료수: 50,614개

cntPerPage = 100
totalCnt = 50614 #한국어 논문 개수
pageCnt = totalCnt // cntPerPage + 1

print(f"총 자료: {totalCnt}")
print(f"총 페이지: {pageCnt}")

#브라우저 생성 및 사이트 이동
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)

prepare_download()
#page는 0부터 시작함
for pageIndex in range(62, pageCnt):
    startCnt = pageIndex * 100
    print(f'[INFO] {pageIndex + 1} page 수집합니다( goPage({startCnt}) )')
    if startCnt != 0 :
        browser.execute_script(f'goPage({startCnt})')
    download_abstract(pageIndex, wait)

#브라우저 종료
browser.close()

In [ ]:
#csv 파일을 Merge 
list_of_files = [file for file in glob(f'{DOWNLOAD_DIR}\\csv\\*.csv', recursive=False)]
result = pd.concat([pd.read_csv(file) for file in list_of_files])

oneCsvFileName = os.path.join(DOWNLOAD_DIR, "abstracts.csv")
result.to_csv(oneCsvFileName, index=True)

#총 50,614 개

In [ ]:
list_of_files = [file for file in glob(f'{DOWNLOAD_DIR}\\csv\\*.csv', recursive=False)]
len(list_of_files)